In [1]:
import matplotlib.pyplot as plt 
import numpy as np

In [2]:
#Punto 4

In [3]:
#Punto 5

In [4]:
#Punto 7
def sor(A,b,x0,w,itmax=1000,tolerancia=1e-14):
    x=x0.copy()
    sum_k= x.copy()
    residuo = np.linalg.norm( np.dot(A,x) - b)
    it=0
    while residuo>= tolerancia and it < itmax:
    
        sum_k[:]=0
        
        for i in range (A.shape[0]):
            for j in range (A.shape[1]):
                if i != j:
                     sum_k[i] += A[i][j] * x[j]
            x[i] = (1 - w) * x[i] + (w / A[i][i]) * (b[i] - sum_k[i])
        
        residuo = np.linalg.norm(np.dot(A, x) - b)
        it += 1
        
        if residuo > 1000:
            print('No calculable con SOR')
            x[:] = 0.
            break
    
        
    return x,it
def omega_optimo(A,b,x0,itmax=1000,tolerancia=1e-14):
    w=None
    iteracion_minima=float('inf')
    for w0 in range(1,2):
        metodo= sor(A,b,x0,itmax,tolerancia)
        iteracion=len(metodo)
        
        if iteracion< iteracion_minima:
            iteracion_minima=iteracion
            w=w0   
    return w
A = np.array([[3,-1,-1],[-1.,3.,1.],[2,1,4]])
b = np.array([1.,3.,7.])
w=omega_optimo(A,b,np.array([0,0,0]))
x=sor(A,b,np.array([0,0,0]),w)
x

No calculable con SOR


(array([1, 1, 1]), 2)

In [14]:
# punto 11 

H= np.array([[1,2,-1],[1,0,1],[4,-4,5]])


def autovalor_min (m):
    W = np.linalg.inv(m)
    
    q = np.random.rand(len(W))
    n=np.shape(W)[0]
    
    q1=np.zeros(shape=(n,1))
    for i in range(len(q)):
        q1[i][0]= q[i]
        
    for i in range(100):
        z= np.dot(W, q1)
        q1= z/np.sqrt(np.dot(z.T,z))
        valor_min= np.dot(q1.T,W)
        valor_min= np.dot(valor_min, q1)
    return "El valor de estado base E0 es "+ str(1/valor_min) + " y su vector propio es " + str(q1).replace("\n", " ")
autovalor_min(H)


'El valor de estado base E0 es [[1.]] y su vector propio es [[-0.40824829]  [ 0.40824829]  [ 0.81649658]]'